# Aracne Setup
Aracne uses information theoretic approaches for constructing gene regulatory networks using gene expression data. 

In [9]:
import os
import pandas


class FilePaths():
    def __init__(self):
        self.dire = r'/home/b3053674/Documents/Miscellaneous/Aracne/AracneMicroarray'
        self.input_file = os.path.join(self.dire, 'MicroarrayDEGs.csv')
        self.output_filename = os.path.join(self.dire, 'ArcneOutput.adj')
        self.TFlist_file=os.path.join(self.dire, 'TFsInMicroarrayDataProbeIDs.txt')
        
        
        
        
F=FilePaths()


Get annotation from input file

In [24]:
data = pandas.read_csv(F.input_file, sep='\t')
anno_df = data[['ProbeID','GeneSymbol']]

print(anno_df)

           ProbeID GeneSymbol
0    11746909_a_at       A1CF
1    11736238_a_at      ABCA5
2      11724734_at      ABCB8
3      11723976_at      ABCC8
4    11718612_a_at      ABCD4
5    11758217_s_at    ABHD17C
6    11744541_a_at      ACKR3
7      11730931_at      ACSS3
8      11738454_at     ACTBL2
9      11727682_at     ACVR2B
10   11733762_a_at   ADAMTSL1
11   11756406_x_at        ADM
12   11743232_a_at       AFDN
13   11743984_a_at      AKAP1
14   11723804_a_at     AKAP12
15   11716720_a_at     AKT1S1
16     11733006_at     AMOTL2
17   11719576_s_at        AMT
18   11719577_a_at        AMT
19   11755899_x_at   ANKRD36B
20   11755987_a_at    ANKRD44
21   11756425_a_at     ANKS1A
22   11727683_a_at      AP3B2
23   11734819_s_at      APBB2
24   11723975_x_at      APOL3
25   11722736_s_at      APPL1
26   11733928_a_at      ARMC8
27   11751585_a_at       ASB8
28   11716966_a_at       ATN1
29   11716154_x_at    B4GALT1
..             ...        ...
197  11763601_x_at      WDR62
198  11746

In [49]:
data = pandas.read_csv(F.output_filename, sep='\t', skiprows=18,index_col=0,header=None)

# print(data.head())
anno_df2=anno_df.set_index('ProbeID')
# print(anno_df)

filtered_anno_df = anno_df2[anno_df2.index.isin(list(data.index))  ] 

merged =pandas.merge(data, anno_df2, left_index=True, right_index=True)


merged2=merged.reset_index()
# print(merged2)
merged3=merged2.set_index(['index','GeneSymbol'])
print(merged3)

                                    1         2              3         4    \
index         GeneSymbol                                                     
11746909_a_at A1CF        11736238_a_at  0.038230    11724734_at  0.024966   
11736238_a_at ABCA5       11746909_a_at  0.038230    11724734_at  0.024771   
11724734_at   ABCB8       11746909_a_at  0.024966  11736238_a_at  0.024771   
11723976_at   ABCC8       11746909_a_at  0.017006  11736238_a_at  0.046125   
11718612_a_at ABCD4       11746909_a_at  0.014982  11736238_a_at  0.050172   
11758217_s_at ABHD17C     11746909_a_at  0.033042  11736238_a_at  0.060155   
11744541_a_at ACKR3       11746909_a_at  0.025869  11736238_a_at  0.039581   
11730931_at   ACSS3       11746909_a_at  0.019453  11736238_a_at  0.020194   
11738454_at   ACTBL2      11746909_a_at  0.029000  11736238_a_at  0.029538   
11727682_at   ACVR2B      11746909_a_at  0.054013  11736238_a_at  0.057907   
11733762_a_at ADAMTSL1    11746909_a_at  0.024412  11736238_a_at

In [59]:
merged3 = merged3.replace(merged3.reset_index(level=1)['GeneSymbol'])

In [136]:
def create_network_table(data):
    """
    
    
    ## sanity check 
                                 Interactant    Weight Interactant    Weight  \
    index         GeneSymbol                                               
    11746909_a_at A1CF             ABCA5  0.038230       ABCB8  0.024966   
    11736238_a_at ABCA5             A1CF  0.038230       ABCB8  0.024771  
    """
    
    col = ['Interactant','Weight']*int(data.shape[1]/2) 
    data.columns = col
    
    interactions = data['Interactant'].stack()
    weights = data['Weight'].stack()

    interactions.index = interactions.index.droplevel(2)
    weights.index = weights.index.droplevel(2)
    
#     print (data)
    

    weights = pandas.DataFrame(weights)
    interactions = pandas.DataFrame(interactions)
    interactions.columns = ['interactions']
    weights.columns = ['weights']
    df  = interactions.merge(weights, left_index=True, right_index=True)     
    df = df.sort_index(level=1)
#     df = df.sort_values(by = 'weights')
    df = df.reset_index(level=1)
    df = df[df['GeneSymbol']!='---']
    df = df.reset_index()
    
    return df
    
interactions = create_network_table(merged3)



In [149]:
import numpy

# print(interactions)
# print(interactions.set_index('GeneSymbol','interactions').reset_index())

# print(interactions.groupby(by = ['GeneSymbol','interactions']).aggregate(numpy.mean))
df_dct = {}
for label, df in interactions.groupby(by = ['GeneSymbol', 'interactions']):
    label, df['weights'].mean())

('A1CF', '---') 0.0301887557522
('A1CF', 'ABCA5') 0.0301887557522
('A1CF', 'ABCB8') 0.0301887557522
('A1CF', 'ABCC8') 0.0301887557522
('A1CF', 'ABCD4') 0.0301887557522
('A1CF', 'ABHD17C') 0.0301887557522
('A1CF', 'ACKR3') 0.0301887557522
('A1CF', 'ACSS3') 0.0301887557522
('A1CF', 'ACTBL2') 0.0301887557522
('A1CF', 'ACVR2B') 0.0301887557522
('A1CF', 'ADAMTSL1') 0.0301887557522
('A1CF', 'ADM') 0.0301887557522
('A1CF', 'AFDN') 0.0301887557522
('A1CF', 'AKAP1') 0.0301887557522
('A1CF', 'AKAP12') 0.0301887557522
('A1CF', 'AKT1S1') 0.0301887557522
('A1CF', 'AMOTL2') 0.0301887557522
('A1CF', 'AMT') 0.0301887557522
('A1CF', 'ANKRD36B') 0.0301887557522
('A1CF', 'ANKRD44') 0.0301887557522
('A1CF', 'ANKS1A') 0.0301887557522
('A1CF', 'AP3B2') 0.0301887557522
('A1CF', 'APBB2') 0.0301887557522
('A1CF', 'APOL3') 0.0301887557522
('A1CF', 'APPL1') 0.0301887557522
('A1CF', 'ARMC8') 0.0301887557522
('A1CF', 'ASB8') 0.0301887557522
('A1CF', 'ATN1') 0.0301887557522
('A1CF', 'B4GALT1') 0.0301887557522
('A1C

('ABCD4', 'ACKR3') 0.0839106075221
('ABCD4', 'ACSS3') 0.0839106075221
('ABCD4', 'ACTBL2') 0.0839106075221
('ABCD4', 'ACVR2B') 0.0839106075221
('ABCD4', 'ADAMTSL1') 0.0839106075221
('ABCD4', 'ADM') 0.0839106075221
('ABCD4', 'AFDN') 0.0839106075221
('ABCD4', 'AKAP1') 0.0839106075221
('ABCD4', 'AKAP12') 0.0839106075221
('ABCD4', 'AKT1S1') 0.0839106075221
('ABCD4', 'AMOTL2') 0.0839106075221
('ABCD4', 'AMT') 0.0839106075221
('ABCD4', 'ANKRD36B') 0.0839106075221
('ABCD4', 'ANKRD44') 0.0839106075221
('ABCD4', 'ANKS1A') 0.0839106075221
('ABCD4', 'AP3B2') 0.0839106075221
('ABCD4', 'APBB2') 0.0839106075221
('ABCD4', 'APOL3') 0.0839106075221
('ABCD4', 'APPL1') 0.0839106075221
('ABCD4', 'ARMC8') 0.0839106075221
('ABCD4', 'ASB8') 0.0839106075221
('ABCD4', 'ATN1') 0.0839106075221
('ABCD4', 'B4GALT1') 0.0839106075221
('ABCD4', 'BACE1') 0.0839106075221
('ABCD4', 'BCL7A') 0.0839106075221
('ABCD4', 'BRF1') 0.0839106075221
('ABCD4', 'BTG2') 0.0839106075221
('ABCD4', 'C9orf72') 0.0839106075221
('ABCD4', '

('ACSS3', 'F3') 0.0335378917257
('ACSS3', 'FAM161B') 0.0335378917257
('ACSS3', 'FAM170A') 0.0335378917257
('ACSS3', 'FANCD2') 0.0335378917257
('ACSS3', 'FBXW2') 0.0335378917257
('ACSS3', 'FER') 0.0335378917257
('ACSS3', 'FJX1') 0.0335378917257
('ACSS3', 'FZD7') 0.0335378917257
('ACSS3', 'GCNT1') 0.0335378917257
('ACSS3', 'GLUD1') 0.0335378917257
('ACSS3', 'GMFG') 0.0335378917257
('ACSS3', 'GOLGA7B') 0.0335378917257
('ACSS3', 'HAUS5') 0.0335378917257
('ACSS3', 'HES1') 0.0335378917257
('ACSS3', 'HIVEP3') 0.0335378917257
('ACSS3', 'HLA-G') 0.0335378917257
('ACSS3', 'HOXB13') 0.0335378917257
('ACSS3', 'HOXC6') 0.0335378917257
('ACSS3', 'HYKK') 0.0335378917257
('ACSS3', 'HYOU1') 0.0335378917257
('ACSS3', 'ICAM4') 0.0335378917257
('ACSS3', 'ID1') 0.0335378917257
('ACSS3', 'INMT') 0.0335378917257
('ACSS3', 'INPP5D') 0.0335378917257
('ACSS3', 'IP6K1') 0.0335378917257
('ACSS3', 'IPPK') 0.0335378917257
('ACSS3', 'JAZF1') 0.0335378917257
('ACSS3', 'KCNJ5') 0.0335378917257
('ACSS3', 'KCNS3') 0.033

('ADAMTSL1', 'KMT2D') 0.0353865265044
('ADAMTSL1', 'KRTAP12-2') 0.0353865265044
('ADAMTSL1', 'LCMT2') 0.0353865265044
('ADAMTSL1', 'LDB3') 0.0353865265044
('ADAMTSL1', 'LIF') 0.0353865265044
('ADAMTSL1', 'MAFG') 0.0353865265044
('ADAMTSL1', 'MTG2') 0.0353865265044
('ADAMTSL1', 'MYO10') 0.0353865265044
('ADAMTSL1', 'NAIP') 0.0353865265044
('ADAMTSL1', 'NEO1') 0.0353865265044
('ADAMTSL1', 'NIPAL4') 0.0353865265044
('ADAMTSL1', 'NNT') 0.0353865265044
('ADAMTSL1', 'NUDT14') 0.0353865265044
('ADAMTSL1', 'NUDT6') 0.0353865265044
('ADAMTSL1', 'P2RX2') 0.0353865265044
('ADAMTSL1', 'PARP2') 0.0353865265044
('ADAMTSL1', 'PDK1') 0.0353865265044
('ADAMTSL1', 'PIGA') 0.0353865265044
('ADAMTSL1', 'PLEKHA2') 0.0353865265044
('ADAMTSL1', 'PLEKHA6') 0.0353865265044
('ADAMTSL1', 'PMEPA1') 0.0353865265044
('ADAMTSL1', 'PMM2') 0.0353865265044
('ADAMTSL1', 'PPM1E') 0.0353865265044
('ADAMTSL1', 'PRICKLE2') 0.0353865265044
('ADAMTSL1', 'PSPC1') 0.0353865265044
('ADAMTSL1', 'PXDC1') 0.0353865265044
('ADAMTSL1

('AKAP1', 'FANCD2') 0.0393758201327
('AKAP1', 'FBXW2') 0.0393758201327
('AKAP1', 'FER') 0.0393758201327
('AKAP1', 'FJX1') 0.0393758201327
('AKAP1', 'FZD7') 0.0393758201327
('AKAP1', 'GCNT1') 0.0393758201327
('AKAP1', 'GLUD1') 0.0393758201327
('AKAP1', 'GMFG') 0.0393758201327
('AKAP1', 'GOLGA7B') 0.0393758201327
('AKAP1', 'HAUS5') 0.0393758201327
('AKAP1', 'HES1') 0.0393758201327
('AKAP1', 'HIVEP3') 0.0393758201327
('AKAP1', 'HLA-G') 0.0393758201327
('AKAP1', 'HOXB13') 0.0393758201327
('AKAP1', 'HOXC6') 0.0393758201327
('AKAP1', 'HYKK') 0.0393758201327
('AKAP1', 'HYOU1') 0.0393758201327
('AKAP1', 'ICAM4') 0.0393758201327
('AKAP1', 'ID1') 0.0393758201327
('AKAP1', 'INMT') 0.0393758201327
('AKAP1', 'INPP5D') 0.0393758201327
('AKAP1', 'IP6K1') 0.0393758201327
('AKAP1', 'IPPK') 0.0393758201327
('AKAP1', 'JAZF1') 0.0393758201327
('AKAP1', 'KCNJ5') 0.0393758201327
('AKAP1', 'KCNS3') 0.0393758201327
('AKAP1', 'KCTD15') 0.0393758201327
('AKAP1', 'KDM4B') 0.0393758201327
('AKAP1', 'KDM5A') 0.039

...

('WBSCR17', 'SLC34A1') 0.0314761890265
('WBSCR17', 'SLC7A1') 0.0314761890265
('WBSCR17', 'SMCO4') 0.0314761890265
('WBSCR17', 'SMYD4') 0.0314761890265
('WBSCR17', 'SNAPC4') 0.0314761890265
('WBSCR17', 'SORBS1') 0.0314761890265
('WBSCR17', 'SPEN') 0.0314761890265
('WBSCR17', 'SRBD1') 0.0314761890265
('WBSCR17', 'SRC') 0.0314761890265
('WBSCR17', 'STAG3') 0.0314761890265
('WBSCR17', 'SUSD6') 0.0314761890265
('WBSCR17', 'SYP') 0.0314761890265
('WBSCR17', 'TACC2') 0.0314761890265
('WBSCR17', 'TCN2') 0.0314761890265
('WBSCR17', 'TLR4') 0.0314761890265
('WBSCR17', 'TMEM170B') 0.0314761890265
('WBSCR17', 'TMEM2') 0.0314761890265
('WBSCR17', 'TMEM25') 0.0314761890265
('WBSCR17', 'TMEM53') 0.0314761890265
('WBSCR17', 'TNC') 0.0314761890265
('WBSCR17', 'TPD52') 0.0314761890265
('WBSCR17', 'TPP1') 0.0314761890265
('WBSCR17', 'TRIM10') 0.0314761890265
('WBSCR17', 'TRPS1') 0.0314761890265
('WBSCR17', 'TRUB1') 0.0314761890265
('WBSCR17', 'TSC22D1') 0.0314761890265
('WBSCR17', 'TUBGCP3') 0.0314761890

('ZFPM1', 'GOLGA7B') 0.0359405340708
('ZFPM1', 'HAUS5') 0.0359405340708
('ZFPM1', 'HES1') 0.0359405340708
('ZFPM1', 'HIVEP3') 0.0359405340708
('ZFPM1', 'HLA-G') 0.0359405340708
('ZFPM1', 'HOXB13') 0.0359405340708
('ZFPM1', 'HOXC6') 0.0359405340708
('ZFPM1', 'HYKK') 0.0359405340708
('ZFPM1', 'HYOU1') 0.0359405340708
('ZFPM1', 'ICAM4') 0.0359405340708
('ZFPM1', 'ID1') 0.0359405340708
('ZFPM1', 'INMT') 0.0359405340708
('ZFPM1', 'INPP5D') 0.0359405340708
('ZFPM1', 'IP6K1') 0.0359405340708
('ZFPM1', 'IPPK') 0.0359405340708
('ZFPM1', 'JAZF1') 0.0359405340708
('ZFPM1', 'KCNJ5') 0.0359405340708
('ZFPM1', 'KCNS3') 0.0359405340708
('ZFPM1', 'KCTD15') 0.0359405340708
('ZFPM1', 'KDM4B') 0.0359405340708
('ZFPM1', 'KDM5A') 0.0359405340708
('ZFPM1', 'KIAA0586') 0.0359405340708
('ZFPM1', 'KIAA0930') 0.0359405340708
('ZFPM1', 'KIF2C') 0.0359405340708
('ZFPM1', 'KIFC3') 0.0359405340708
('ZFPM1', 'KLF11') 0.0359405340708
('ZFPM1', 'KLHL26') 0.0359405340708
('ZFPM1', 'KMT2D') 0.0359405340708
('ZFPM1', 'KR

('ZNF260', 'TPD52') 0.0628893617257
('ZNF260', 'TPP1') 0.0628893617257
('ZNF260', 'TRIM10') 0.0628893617257
('ZNF260', 'TRPS1') 0.0628893617257
('ZNF260', 'TRUB1') 0.0628893617257
('ZNF260', 'TSC22D1') 0.0628893617257
('ZNF260', 'TUBGCP3') 0.0628893617257
('ZNF260', 'TXNIP') 0.0628893617257
('ZNF260', 'VSTM1') 0.0628893617257
('ZNF260', 'VWA5A') 0.0628893617257
('ZNF260', 'WBSCR17') 0.0628893617257
('ZNF260', 'WDR62') 0.0628893617257
('ZNF260', 'WIZ') 0.0628893617257
('ZNF260', 'XPO1') 0.0628893617257
('ZNF260', 'ZFPM1') 0.0628893617257
('ZNF260', 'ZFYVE27') 0.0628893617257
('ZNF260', 'ZNF181') 0.0628893617257
('ZNF260', 'ZNF33A') 0.0628893617257
('ZNF260', 'ZNF385A') 0.0628893617257
('ZNF260', 'ZNF436') 0.0628893617257
('ZNF260', 'ZNF469') 0.0628893617257
('ZNF260', 'ZNF496') 0.0628893617257
('ZNF260', 'ZNF512') 0.0628893617257
('ZNF260', 'ZNF516') 0.0628893617257
('ZNF260', 'ZNF560') 0.0628893617257
('ZNF260', 'ZNF71') 0.0628893617257
('ZNF33A', '---') 0.051946710354
('ZNF33A', 'A1CF

('ZNF436', 'SCARA5') 0.079450735531
('ZNF436', 'SECTM1') 0.079450735531
('ZNF436', 'SERPINE1') 0.079450735531
('ZNF436', 'SH3GLB2') 0.079450735531
('ZNF436', 'SHANK2') 0.079450735531
('ZNF436', 'SHPRH') 0.079450735531
('ZNF436', 'SHTN1') 0.079450735531
('ZNF436', 'SIX1') 0.079450735531
('ZNF436', 'SIX5') 0.079450735531
('ZNF436', 'SLC15A1') 0.079450735531
('ZNF436', 'SLC30A1') 0.079450735531
('ZNF436', 'SLC34A1') 0.079450735531
('ZNF436', 'SLC7A1') 0.079450735531
('ZNF436', 'SMCO4') 0.079450735531
('ZNF436', 'SMYD4') 0.079450735531
('ZNF436', 'SNAPC4') 0.079450735531
('ZNF436', 'SORBS1') 0.079450735531
('ZNF436', 'SPEN') 0.079450735531
('ZNF436', 'SRBD1') 0.079450735531
('ZNF436', 'SRC') 0.079450735531
('ZNF436', 'STAG3') 0.079450735531
('ZNF436', 'SUSD6') 0.079450735531
('ZNF436', 'SYP') 0.079450735531
('ZNF436', 'TACC2') 0.079450735531
('ZNF436', 'TCN2') 0.079450735531
('ZNF436', 'TLR4') 0.079450735531
('ZNF436', 'TMEM170B') 0.079450735531
('ZNF436', 'TMEM2') 0.079450735531
('ZNF436'

('ZNF516', 'HYKK') 0.038595075177
('ZNF516', 'HYOU1') 0.038595075177
('ZNF516', 'ICAM4') 0.038595075177
('ZNF516', 'ID1') 0.038595075177
('ZNF516', 'INMT') 0.038595075177
('ZNF516', 'INPP5D') 0.038595075177
('ZNF516', 'IP6K1') 0.038595075177
('ZNF516', 'IPPK') 0.038595075177
('ZNF516', 'JAZF1') 0.038595075177
('ZNF516', 'KCNJ5') 0.038595075177
('ZNF516', 'KCNS3') 0.038595075177
('ZNF516', 'KCTD15') 0.038595075177
('ZNF516', 'KDM4B') 0.038595075177
('ZNF516', 'KDM5A') 0.038595075177
('ZNF516', 'KIAA0586') 0.038595075177
('ZNF516', 'KIAA0930') 0.038595075177
('ZNF516', 'KIF2C') 0.038595075177
('ZNF516', 'KIFC3') 0.038595075177
('ZNF516', 'KLF11') 0.038595075177
('ZNF516', 'KLHL26') 0.038595075177
('ZNF516', 'KMT2D') 0.038595075177
('ZNF516', 'KRTAP12-2') 0.038595075177
('ZNF516', 'LCMT2') 0.038595075177
('ZNF516', 'LDB3') 0.038595075177
('ZNF516', 'LIF') 0.038595075177
('ZNF516', 'MAFG') 0.038595075177
('ZNF516', 'MTG2') 0.038595075177
('ZNF516', 'MYO10') 0.038595075177
('ZNF516', 'NAIP'